<a href="https://colab.research.google.com/github/hammaad2002/CenterFusionModel/blob/main/CenterFusion%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!git clone --recursive https://github.com/brandesjj/centerfusionpp
!pip install torch torchvision torchaudio nuscenes-devkit onnx progress
%cd /content/centerfusionpp/src/lib/model/networks
!git clone https://github.com/jinfagang/DCNv2_latest
#After this step rename the folder 'DCNv2_latest' to just 'DCNv2'
!mv /content/centerfusionpp/src/lib/model/networks/DCNv2_latest /content/centerfusionpp/src/lib/model/networks/DCNv2
# bulding DCNv2 model
%cd /content/centerfusionpp/src/lib/model/networks/DCNv2
! ./make.sh

In [ ]:
!python -V

Python 3.10.12


onnx file

```
'''
Script to convert a trained CenterNet model to ONNX, currently only
support non-DCN models.
'''
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import json
import cv2
import numpy as np
import time
from progress.bar import Bar
import torch
import copy

from model.model import create_model, load_model
from opts import opts
from dataset.dataset_factory import dataset_factory
from detector import Detector


def convert_onnx(opt):
  os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
  opt.model_output_list = True
  if opt.gpus[0] >= 0:
    opt.device = torch.device('cuda')
  else:
    opt.device = torch.device('cpu')
  Dataset = dataset_factory[opt.test_dataset]
  opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
  print('-------')
  print(opt)
  print(opt.input_h)
  print(opt.input_w)
  print(opt.exp_id)
  print('-------')
  model = create_model(
      opt.arch, opt.heads, opt.head_conv, opt=opt)
  if opt.load_model != '':
    model = load_model(model, opt.load_model, opt)

  opt.device = 'cpu'
  model = model.to(opt.device)
  model.eval()
  dummy_input1 = torch.randn(1, 3, opt.input_h, opt.input_w).to(opt.device)
  storing_path = '/content/onnx_models/'
  print(opt.task)
  print('-dsardawd')
  if opt.tracking:
    dummy_input2 = torch.randn(1, 3, opt.input_h, opt.input_w).to(opt.device)
    if opt.pre_hm:
      dummy_input3 = torch.randn(1, 1, opt.input_h, opt.input_w).to(opt.device)
      torch.onnx.export(
        model, (dummy_input1, dummy_input2, dummy_input3),
        "../models/{}.onnx".format(opt.exp_id))
    else:
      torch.onnx.export(
        model, (dummy_input1, dummy_input2),
        storing_path + opt.task + '.onnx')
  else:
    torch.onnx.export(
      model, (dummy_input1, ),
      storing_path + opt.task  + '.onnx')
if __name__ == '__main__':
  opt = opts().parse()
  convert_onnx(opt)
```





/content/centerfusionpp/src/lib/model/networks/base_model.py file

At line 202
```
      x = torch.cat((batch['image'], early_fusion_norm), 1)

    else:    
      # Input to backbone is RGB image
      print('-------')
      print(type(batch))
      print(batch)
      print(batch.shape)
      # print(batch['image'].shape)
      print('-------')
      x = batch
```



In [ ]:
!mkdir /content/onnx_models/

In [66]:
!python3 /content/centerfusionpp/src/convert_onnx.py ctdet

DCN import in dla.py successful!
DCN import in resdcn.py successful!
DCN import in necks/dlaup.py successful!
DCN import in necks/msraup.py successful!
Fix size testing.
Training chunk_sizes (master, slave1, slave2, ...):  [32]
Input height: 448  & width:  800
Heads:  {'hm': 10, 'reg': 2, 'wh': 2}
Weights:  {'hm': 1, 'reg': 1, 'wh': 0.1}
Convolutional setup per head:  {'hm': [256], 'reg': [256], 'wh': [256]}
-------
Namespace(task='ctdet', dataset='nuscenes', test_dataset='nuscenes', exp_id='default', eval=False, debug=0, eval_frustum=0, no_pause=False, demo='', load_model='', resume=False, gpus=[0], num_workers=4, not_cuda_benchmark=False, seed=317, not_set_cuda_env=False, print_iter=0, save_all=False, vis_thresh=0.3, debugger_theme='white', run_dataset_eval=False, save_imgs=[], save_img_suffix='', skip_first=-1, save_video=False, save_framerate=30, resize_video=False, video_h=512, video_w=512, transpose_video=False, show_track_color=False, not_show_bbox=False, not_show_number=False, 

In [ ]:
!pip install --upgrade onnx onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import onnx
onnx.checker.check_model('/content/onnx_models/ctdet.onnx', full_check=True)
print("Done.")

Done.


In [65]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00


In [67]:
import onnxruntime as ort
import numpy as np
import torch

# Load the ONNX model
model = onnx.load('/content/onnx_models/ctdet.onnx')

# Create an inference session using ONNX Runtime
sess = ort.InferenceSession('/content/onnx_models/ctdet.onnx')

# Assuming you have input data in a numpy array named 'input_data'
input_data = torch.randn(1, 3, 448, 800)

# Run the model (replace 'input_name' and 'output_name' with the actual names)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

result = sess.run([output_name], {input_name: input_data})

print(result)

Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from /content/onnx_models/ctdet.onnx failed:Fatal error: domain_version:Plugin(-1) is not a registered function/op

In [ ]:
!pip install netron

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.0 MB/s eta 0:00:00


In [ ]:
import netron
import IPython

# import netron
# netron.start('test.h5', 8081) for running locally

netron.start('/content/onnx_models/ctdet.onnx', 6006, browse=False)

# IPython.display.IFrame(f"http://localhost:{6006}", width=1000, height=1000) this isn't working either :(

Stopping http://localhost:6006
Serving '/content/onnx_models/ctdet.onnx' at http://localhost:6006


In order to make the conversion work till the above progress:

1) Replace all the files that import DCN model using the following lines:


```
# from .DCNv2.dcn_v2 import DCN
from .DCNv2.dcn_v2_onnx import DCN
```
2) In the dcn_v2_onnx file as the following code just after the import:

```
import _ext as _backend

# Define the custom operator registration function
def my_custom_op(g, input, offset_mask, weight, bias, stride, padding, dilation, deformable_groups):
    # This should match the definition in the symbolic method of your custom operator class.
    return g.op("domain_version::Plugin", input, offset_mask, weight, bias,
                stride_s=stride, padding_s=padding, dilation_s=dilation, deformable_groups_i=deformable_groups)
```

3) And in the __init__ of the same file for class 'DCNv2' and '' add the following line:



```
# Register the custom operator with a unique domain name.
register_custom_op_symbolic('domain_version::Plugin', my_custom_op, 17)
```

4) And finally in the convert_onnx.py file change the export to:

```
torch.onnx.export(
model, (dummy_input1, ),
storing_path + opt.task  + '.onnx',
opset_version=17,
custom_opsets={"domain_version": 17},
verbose=False)
```




# My latest work which is to convert the same model but using torhc.onnx.dynamo approach which might fix the warning being generated above:

```
'''
Script to convert a trained CenterNet model to ONNX, currently only
support non-DCN models.
'''
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import os
import json
import cv2
import numpy as np
import time
from progress.bar import Bar
import torch
import copy

from model.model import create_model, load_model
from opts import opts
from dataset.dataset_factory import dataset_factory
from detector import Detector
from torch.onnx import OnnxRegistry
import onnxscript

# custom_aten = onnxscript.values.Opset(domain="custom.aten", version=1)

# # Define the custom operator registration function
# def my_custom_op(g, input, offset_mask, weight, bias, stride, padding, dilation, deformable_groups):
#     # This should match the definition in the symbolic method of your custom operator class.
#     return g.op("domain_version::Plugin", input, offset_mask, weight, bias,
#                 stride_s=stride, padding_s=padding, dilation_s=dilation, deformable_groups_i=deformable_groups)

def convert_onnx(opt):
  os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
  opt.model_output_list = True
  if opt.gpus[0] >= 0:
    opt.device = torch.device('cuda')
  else:
    opt.device = torch.device('cpu')
  Dataset = dataset_factory[opt.test_dataset]
  opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
  # print('-------')
  # print(opt)
  # print(opt.input_h)
  # print(opt.input_w)
  # print(opt.exp_id)
  # print('-------')
  model = create_model(
      opt.arch, opt.heads, opt.head_conv, opt=opt)
  if opt.load_model != '':
    model = load_model(model, opt.load_model, opt)

  opt.device = 'cpu'
  model = model.to(opt.device)
  model.eval()
  dummy_input1 = torch.randn(1, 3, opt.input_h, opt.input_w).to(opt.device)
  storing_path = '/content/onnx_models/'
  # print(opt.task)
  if opt.tracking:
    dummy_input2 = torch.randn(1, 3, opt.input_h, opt.input_w).to(opt.device)
    if opt.pre_hm:
      dummy_input3 = torch.randn(1, 1, opt.input_h, opt.input_w).to(opt.device)
      torch.onnx.export(
        model, (dummy_input1, dummy_input2, dummy_input3),
        "../models/{}.onnx".format(opt.exp_id))
    else:
      torch.onnx.export(
        model, (dummy_input1, dummy_input2),
        storing_path + opt.task + '.onnx')
  else:
    
    torch.onnx.export(
      model, (dummy_input1, ),
      storing_path + opt.task  + '.onnx',
      opset_version=17,
      custom_opsets={"domain_version": 17},
      verbose=False)
    # temp = OnnxRegistry()
    # temp.register_op(function=my_custom_op, namespace='domain_version',
    #                               op_name='Plugin')
    # export = torch.onnx.ExportOptions(onnx_registry=temp)
    # torch.onnx.dynamo_export(
    # model, (dummy_input1, ),
    # export_options=export,
    # ).save("my_simple_model.onnx")

if __name__ == '__main__':
  opt = opts().parse()
  convert_onnx(opt)
```





```
#!/usr/bin/env python
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import math
import torch
from torch import nn
from torch.autograd import Function
from torch.nn.modules.utils import _pair
from torch.autograd.function import once_differentiable
import json
from torch.onnx import register_custom_op_symbolic
from torch.onnx import OnnxRegistry

import _ext as _backend

# Define the custom operator registration function
def my_custom_op(g, input, offset_mask, weight, bias, stride, padding, dilation, deformable_groups):
    # This should match the definition in the symbolic method of your custom operator class.
    return g.op("domain_version::Plugin", input, offset_mask, weight, bias,
                stride_s=stride, padding_s=padding, dilation_s=dilation, deformable_groups_i=deformable_groups)

class _DCNv2(Function):

    @staticmethod
    def symbolic(g, input, offset_mask, weight, bias, stride, padding, dilation, deformable_groups):
        return g.op("domain_version::Plugin", input, offset_mask, weight, bias, name_s="DCNv2", info_s=json.dumps({
            "dilation": dilation,
            "padding": padding,
            "stride": stride,
            "deformable_groups": deformable_groups
        }))

    ###############################################  修改的部分 ############################################################
    # 这里以下的修改，并不是必须的，仅仅是复现DCN的时候，输入改成是input和offset_mask，原始的做法是chunk分割为x, y, mask，
    # 然后再cat，再对mask做sigmoid后输入到dcn。这么做效率比较底下。实现DCN的时候输入input和offset_mask，内部做了分割和sigmomid
    # 减少数据流转，提高效率，也因此在这里对操作做了修改
    @staticmethod
    def forward(ctx, input, offset_mask, weight, bias,
                stride, padding, dilation, deformable_groups):
        ctx.stride = _pair(stride)
        ctx.padding = _pair(padding)
        ctx.dilation = _pair(dilation)
        ctx.kernel_size = _pair(weight.shape[2:4])
        ctx.deformable_groups = deformable_groups

        o1, o2, mask = torch.chunk(offset_mask, 3, dim=1)
        offset = torch.cat((o1, o2), dim=1)
        mask = torch.sigmoid(mask)

    # @staticmethod
    # def forward(ctx, input, offset, mask, weight, bias,
    #             stride, padding, dilation, deformable_groups):
    #     ctx.stride = _pair(stride)
    #     ctx.padding = _pair(padding)
    #     ctx.dilation = _pair(dilation)
    #     ctx.kernel_size = _pair(weight.shape[2:4])
    #     ctx.deformable_groups = deformable_groups
        output = _backend.dcn_v2_forward(input, weight, bias,
                                         offset, mask,
                                         ctx.kernel_size[0], ctx.kernel_size[1],
                                         ctx.stride[0], ctx.stride[1],
                                         ctx.padding[0], ctx.padding[1],
                                         ctx.dilation[0], ctx.dilation[1],
                                         ctx.deformable_groups)
        ctx.save_for_backward(input, offset, mask, weight, bias)
        return output

    @staticmethod
    @once_differentiable
    def backward(ctx, grad_output):
        input, offset, mask, weight, bias = ctx.saved_tensors
        grad_input, grad_offset, grad_mask, grad_weight, grad_bias = \
            _backend.dcn_v2_backward(input, weight,
                                     bias,
                                     offset, mask,
                                     grad_output,
                                     ctx.kernel_size[0], ctx.kernel_size[1],
                                     ctx.stride[0], ctx.stride[1],
                                     ctx.padding[0], ctx.padding[1],
                                     ctx.dilation[0], ctx.dilation[1],
                                     ctx.deformable_groups)

        return grad_input, grad_offset, grad_mask, grad_weight, grad_bias,\
            None, None, None, None,


dcn_v2_conv = _DCNv2.apply


class DCNv2(nn.Module):

    def __init__(self, in_channels, out_channels,
                 kernel_size, stride=1, padding=1, dilation=1, deformable_groups=1):
        super(DCNv2, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)
        self.padding = _pair(padding)
        self.dilation = _pair(dilation)
        self.deformable_groups = deformable_groups

        self.weight = nn.Parameter(torch.Tensor(
            out_channels, in_channels, *self.kernel_size))
        self.bias = nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()
        # Register the custom operator with a unique domain name.
        register_custom_op_symbolic('domain_version::Plugin', my_custom_op, 17)
        # torch.onnx.OnnxRegistry new line
        # temp = OnnxRegistry()
        # temp.register_op(function=my_custom_op, namespace='domain_version',
        #                           op_name='Plugin')


    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.zero_()

    def forward(self, input, offset, mask):
        assert 2 * self.deformable_groups * self.kernel_size[0] * self.kernel_size[1] == \
            offset.shape[1]
        assert self.deformable_groups * self.kernel_size[0] * self.kernel_size[1] == \
            mask.shape[1]
        return dcn_v2_conv(input, offset, mask,
                           self.weight,
                           self.bias,
                           self.stride,
                           self.padding,
                           self.dilation,
                           self.deformable_groups)


class DCN(DCNv2):

    def __init__(self, in_channels, out_channels,
                 kernel_size, stride, padding,
                 dilation=1, deformable_groups=1):
        super(DCN, self).__init__(in_channels, out_channels,
                                  kernel_size, stride, padding, dilation, deformable_groups)

        channels_ = self.deformable_groups * 3 * self.kernel_size[0] * self.kernel_size[1]
        self.conv_offset_mask = nn.Conv2d(self.in_channels,
                                          channels_,
                                          kernel_size=self.kernel_size,
                                          stride=self.stride,
                                          padding=self.padding,
                                          bias=True)
        self.init_offset()
        
        # Register the custom operator with a unique domain name.
        register_custom_op_symbolic('domain_version::Plugin', my_custom_op, 17)
        # torch.onnx.OnnxRegistry new line
        # temp = OnnxRegistry()
        # temp.register_custom_op(function=my_custom_op, namespace='domain_version',
        #                           op_name='Plugin')
        # export = torch.onnx.ExportOptions(onnx_registry=temp)

    def init_offset(self):
        self.conv_offset_mask.weight.data.zero_()
        self.conv_offset_mask.bias.data.zero_()

    def forward(self, input):
        # out = self.conv_offset_mask(input)
        # o1, o2, mask = torch.chunk(out, 3, dim=1)
        # offset = torch.cat((o1, o2), dim=1)
        # mask = torch.sigmoid(mask)
        # return dcn_v2_conv(input, offset, mask,
        #                    self.weight, self.bias,
        #                    self.stride,
        #                    self.padding,
        #                    self.dilation,
        #                    self.deformable_groups)
        
        ###############################################  修改的部分 ############################################################
        offset_mask = self.conv_offset_mask(input)
        #o1, o2, mask = torch.chunk(out, 3, dim=1)
        #offset = torch.cat((o1, o2), dim=1)
        #mask = torch.sigmoid(mask)
        return dcn_v2_conv(input, offset_mask,
                           self.weight, self.bias,
                           self.stride,
                           self.padding,
                           self.dilation,
                           self.deformable_groups)
        ###############################################  ---------- ############################################################



class _DCNv2Pooling(Function):
    @staticmethod
    def forward(ctx, input, rois, offset,
                spatial_scale,
                pooled_size,
                output_dim,
                no_trans,
                group_size=1,
                part_size=None,
                sample_per_part=4,
                trans_std=.0):
        ctx.spatial_scale = spatial_scale
        ctx.no_trans = int(no_trans)
        ctx.output_dim = output_dim
        ctx.group_size = group_size
        ctx.pooled_size = pooled_size
        ctx.part_size = pooled_size if part_size is None else part_size
        ctx.sample_per_part = sample_per_part
        ctx.trans_std = trans_std

        output, output_count = \
            _backend.dcn_v2_psroi_pooling_forward(input, rois, offset,
                                                  ctx.no_trans, ctx.spatial_scale,
                                                  ctx.output_dim, ctx.group_size,
                                                  ctx.pooled_size, ctx.part_size,
                                                  ctx.sample_per_part, ctx.trans_std)
        ctx.save_for_backward(input, rois, offset, output_count)
        return output

    @staticmethod
    @once_differentiable
    def backward(ctx, grad_output):
        input, rois, offset, output_count = ctx.saved_tensors
        grad_input, grad_offset = \
            _backend.dcn_v2_psroi_pooling_backward(grad_output,
                                                   input,
                                                   rois,
                                                   offset,
                                                   output_count,
                                                   ctx.no_trans,
                                                   ctx.spatial_scale,
                                                   ctx.output_dim,
                                                   ctx.group_size,
                                                   ctx.pooled_size,
                                                   ctx.part_size,
                                                   ctx.sample_per_part,
                                                   ctx.trans_std)

        return grad_input, None, grad_offset, \
            None, None, None, None, None, None, None, None


dcn_v2_pooling = _DCNv2Pooling.apply


class DCNv2Pooling(nn.Module):

    def __init__(self,
                 spatial_scale,
                 pooled_size,
                 output_dim,
                 no_trans,
                 group_size=1,
                 part_size=None,
                 sample_per_part=4,
                 trans_std=.0):
        super(DCNv2Pooling, self).__init__()
        self.spatial_scale = spatial_scale
        self.pooled_size = pooled_size
        self.output_dim = output_dim
        self.no_trans = no_trans
        self.group_size = group_size
        self.part_size = pooled_size if part_size is None else part_size
        self.sample_per_part = sample_per_part
        self.trans_std = trans_std

    def forward(self, input, rois, offset):
        assert input.shape[1] == self.output_dim
        if self.no_trans:
            offset = input.new()
        return dcn_v2_pooling(input, rois, offset,
                              self.spatial_scale,
                              self.pooled_size,
                              self.output_dim,
                              self.no_trans,
                              self.group_size,
                              self.part_size,
                              self.sample_per_part,
                              self.trans_std)


class DCNPooling(DCNv2Pooling):

    def __init__(self,
                 spatial_scale,
                 pooled_size,
                 output_dim,
                 no_trans,
                 group_size=1,
                 part_size=None,
                 sample_per_part=4,
                 trans_std=.0,
                 deform_fc_dim=1024):
        super(DCNPooling, self).__init__(spatial_scale,
                                         pooled_size,
                                         output_dim,
                                         no_trans,
                                         group_size,
                                         part_size,
                                         sample_per_part,
                                         trans_std)

        self.deform_fc_dim = deform_fc_dim

        if not no_trans:
            self.offset_mask_fc = nn.Sequential(
                nn.Linear(self.pooled_size * self.pooled_size *
                          self.output_dim, self.deform_fc_dim),
                nn.ReLU(inplace=True),
                nn.Linear(self.deform_fc_dim, self.deform_fc_dim),
                nn.ReLU(inplace=True),
                nn.Linear(self.deform_fc_dim, self.pooled_size *
                          self.pooled_size * 3)
            )
            self.offset_mask_fc[4].weight.data.zero_()
            self.offset_mask_fc[4].bias.data.zero_()

    def forward(self, input, rois):
        offset = input.new()

        if not self.no_trans:

            # do roi_align first
            n = rois.shape[0]
            roi = dcn_v2_pooling(input, rois, offset,
                                 self.spatial_scale,
                                 self.pooled_size,
                                 self.output_dim,
                                 True,  # no trans
                                 self.group_size,
                                 self.part_size,
                                 self.sample_per_part,
                                 self.trans_std)

            # build mask and offset
            offset_mask = self.offset_mask_fc(roi.view(n, -1))
            offset_mask = offset_mask.view(
                n, 3, self.pooled_size, self.pooled_size)
            o1, o2, mask = torch.chunk(offset_mask, 3, dim=1)
            offset = torch.cat((o1, o2), dim=1)
            mask = torch.sigmoid(mask)

            # do pooling with offset and mask
            return dcn_v2_pooling(input, rois, offset,
                                  self.spatial_scale,
                                  self.pooled_size,
                                  self.output_dim,
                                  self.no_trans,
                                  self.group_size,
                                  self.part_size,
                                  self.sample_per_part,
                                  self.trans_std) * mask
        # only roi_align
        return dcn_v2_pooling(input, rois, offset,
                              self.spatial_scale,
                              self.pooled_size,
                              self.output_dim,
                              self.no_trans,
                              self.group_size,
                              self.part_size,
                              self.sample_per_part,
                              self.trans_std)
```



Code to run c++ code on google colab

```
# !pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
# %load_ext nvcc_plugin
# %%cu
# #include <iostream>
# using namespace std;

# int main() {
#   cout << "Hello World!";
#   return 0;
# }
```

